# Stage 6: Full Memory - Long-Term Memory Tools for Cross-Session Personalization

## Introduction

At the end of Stage 5, we witnessed something interesting: RAMS was quietly extracting facts from every conversation turn and storing them in long-term memory. The compression ratios were impressive! 750 tokens of conversation distilled into 50 tokens of knowledge. But there was a catch. Our agent couldn't *use* any of it.

All that accumulated knowledge of student preferences, past questions, and expressed interests sat in long-term memory. This creates a frustrating scenario:

**Session 1:**  
👤 "I prefer online courses and I'm interested in machine learning."  
🤖 "Great! Let me search for ML courses..."  
*(RAMS extracts and stores: "Student prefers online courses", "Student interested in ML")*

**[User returns the next day and starts a new session]**

**Session 2:**  
👤 "What courses would you recommend for me?"  
🤖 "I'm not sure what you're looking for. Could you tell me your preferences?"  
*(The knowledge exists in long-term memory... but the agent can't reach it)*

The current version of our agent has amnesia despite having a full memory system. Stage 5 gave us the infrastructure, and now we need to give the agent the tools to actually use it.

Once the agent has been completed with long-term memory tools, the architecture will look like this:

```mermaid
graph TD
    Q[Query] --> LM[Load Working Memory]
    LM --> IC[Classify Intent]
    IC -->|GREETING| HG[Handle Greeting]
    IC -->|Other| RA[ReAct Agent]

    subgraph ReAct Loop
        RA --> T1[💭 Thought: Analyze + plan]
        T1 --> A1[🔧 Action: choose tool]
        A1 --> O1[👁️ Observation: Results]
        O1 --> T2[💭 Thought: Evaluate]
        T2 --> |Need more| A1
        T2 --> |Done| F[✅ FINISH]
    end

    subgraph Available Tools
        A1 -->|search| SC[search_courses]
        A1 -->|store| SM[store_memory]
        A1 -->|recall| RM[search_memories]
    end

    F --> SAV[Save Working Memory]
    HG --> SAV
    SAV --> END[Response + Reasoning Trace]

    subgraph Memory Layer
        LM -.->|Read| AMS[(Agent Memory Server)]
        SAV -.->|Write| AMS
        SM -.->|Write| LTM[(Long-term Memory)]
        RM -.->|Read| LTM
    end
```

Now, before diving into managing long-term memory, let's first explore the different types of long-term memory available via the Agent Memory Server. 

### Types of Long-Term Memory

If you look back at the result of the long-term memory query in the previous stage, you'll notice a `memory_type` field indicating how that information was stored: 

```md
CS001 is taught by Allison Hill.
   Topics: education, instructor
   **Type: MemoryTypeEnum.SEMANTIC**
   Created: 2026-01-13 18:31:04.408673+00:00
```

`SEMANTIC` is one of three memory types available in the Agent Memory Server. The other two are episodic, and message. Let's review each of them to explore what their role is and when to use them:

#### Semantic memory

This type of memory (the default for AMS) stores timeless facts and preferences. Things like "Student prefers online courses" or "CS401 requires CS201 as a prerequisite" are examples of semantic memories. They can be user-scoped (personalizing for a specific student) or application-scoped (domain knowledge for everyone). These types of memories are compact and searchable, making them the default choice for most information.

#### Episodic memory

This type of memory captures time-bound events where sequence matters. Things like "Student enrolled in CS101 on 2024-09-15" or "Completed CS101 with grade A" are episodic memories. This type of memory is most useful when the timeline or temporal progression is meaningful.

#### Message memory  

This type of memory stores full conversation snippets where the complete context is crucial. This preserves detailed discussions, nuanced advice, or explanations that would be lost if summarized. However, message memories are token-expensive and should be used sparingly.

<details >  
  <summary> 💡 Click the dropdown to see a few examples of correct memory type decisions </summary>

### Scenario 1: Student States Preference

**User says:** "I prefer online courses because I work during the day."

❌ **Wrong - Message memory (too verbose):**

```python

memory = "Student said: 'I prefer online courses because I work during the day.'"

```

✅ **Right - Semantic memories (extracted facts):**

```python

memory1 = "Student prefers online courses"
memory2 = "Student works during the day"

```

**Why:** Simple facts don't need verbatim storage.

---

### Scenario 2: Course Completion

**User says:** "I just finished CS101 last week!"

❌ **Wrong - Semantic (loses temporal context):**

```python

memory = "Student completed CS101"

```

✅ **Right - Episodic (preserves timeline):**

```python

memory = "Student completed CS101 on 2024-10-20"

```

**Why:** Timeline matters for prerequisites and future planning.

---

### Scenario 3: Complex Career Advice

**Context:** 20-message discussion about career path, including nuanced advice about research vs. industry, application timing, and specific companies to target.

❌ **Wrong - Semantic (loses too much context):**

```python

memory = "Student discussed career planning"

```

✅ **Right - Message memory (preserves full context):**
```python
memory = [Full conversation thread with all nuance]
```

**Why:** Details and context are critical; summary would be inadequate.

</details>

Now that you're familiar with the types of memory, let's begin by setting up our environment and examining the production code that supports this stage.

### Setup

Run the code block below to initialize the agent.

In [ ]:
import sys
from pathlib import Path
from dotenv import load_dotenv

project_root = Path("..").resolve()

stage6_path = project_root / "progressive_agents" / "stage6_full_memory"
src_path = project_root / "src"

load_dotenv(project_root / ".env")

sys.path.insert(0, str(src_path))
sys.path.insert(0, str(stage6_path))

from agent import setup_agent, create_workflow, WorkflowState, get_memory_client, MemoryMessage, WorkingMemory, run_agent_async

print("Initializing Stage 6 Agent...")
course_manager, _ = await setup_agent(auto_load_courses=True)
workflow = create_workflow(course_manager)
print("✅ Agent initialized")

### Implementation Overview

As mentioned, this stage is all about giving our agent the ability to manage long-term memory via tools. In oder for this to happen, we'll build two tools: 

1. `search_memories` - This tool will search the long-term memory for relevant facts, preferences, and past interactions.
2. `store_memory` - This tool will store important information to the student's long-term memory explicitly.

Once the tools are implemented, we'll run a few tests for cross-session personalization by storing preferences in one session and retrieving them in another.

Let's get started.

## Part 1: Implementing Memory Tools

In this first part, we'll build the two tools mentioned above. We'll start by implementing a way for the agent to search long-term memory, giving it the ability to decide when it's relevant to recall information from the past.

### 📌 Task 1: Searching Memories

In order for the agent to be able to search long term memory, we'll need the `search_memories` tool implementation to do the following:

1. Accept a natural language query and optional limit parameter
2. Call the Agent Memory Server's long-term memory search endpoint
3. Return a list of relevant memories formatted for the LLM

In the starter code, we've provided the `SearchMemoriesInput` Pydantic model (similar to the tool input schemas you built in Stage 3) that defines the expected parameters. Your task is to implement the tool function that searches long-term memory and returns relevant results.

<details>
<summary>🛠️ Show Implementation Details</summary>
<br> 
    
**Step 1: Add the Tool Decorator**

Use the `@tool` decorator from LangChain with the `args_schema` parameter set to `SearchMemoriesInput`. This tells LangChain how to parse and validate the tool's inputs.

**Step 2: Validate Student ID**

Check if the `student_id` parameter is provided. If not, return an empty list—we can't search memories without knowing whose memories to search.

**Step 3: Search Memories**

Call the async method `.search_long_term_memory()` on the memory client with these parameters:
- `text`: the search query
- `user_id`: a `UserId` filter with `eq=student_id` (import from `agent_memory_client.filters`)
- `limit`: the maximum number of results

This returns a response object with a `.memories` attribute containing the list of memory objects.

**Step 4: Format Results**

The memory objects returned from Agent Memory Server contain several fields, but we only need a few for the LLM. Convert each memory object to a dictionary with these fields:

- `text`: The actual memory content (e.g., "Student prefers online courses")
- `memory_type`: The type of memory (semantic, episodic, or message—as discussed earlier)
- `topics`: A list of topic tags that were assigned when the memory was stored (e.g., `["preferences", "learning_style"]`). These help categorize memories and can be useful for filtering.

Return the formatted results as a list of these dictionaries.

</details>

In [ ]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from agent_memory_client.filters import UserId


class SearchMemoriesInput(BaseModel):
    """Input schema for searching memories."""
    query: str = Field(
        description="Natural language query to search for in long-term memory. "
        "Examples: 'user preferences', 'completed courses', 'learning goals'"
    )
    limit: int = Field(
        default=3,
        description="Maximum number of memories to return (default: 3)"
    )

# Get the memory client
memory_client = get_memory_client()

@tool(args_schema=SearchMemoriesInput)
async def search_memories(query: str, limit: int = 5, student_id: str = None) -> List[Dict[str, Any]]:
    """
    Search long-term memory for relevant facts and preferences.
    
    Use this tool to recall information from previous conversations,
    such as user preferences, goals, completed courses, or interests.
    
    Args:
        query: Natural language search query
        limit: Maximum number of results to return
        student_id: User identifier (passed via context)
        
    Returns:
        List of relevant memories with text content
    """
    try:
        # Validate student_id
        if not student_id:
            return []
        
        # Search long-term memories
        results = await memory_client.search_long_term_memory(
            text=query,
            user_id=UserId(eq=student_id),
            limit=limit
        )
        
        # Format results for LLM
        formatted_results = [
            {
                "text": memory.text,
                "memory_type": memory.memory_type,
                "topics": memory.topics if hasattr(memory, 'topics') else []
            }
            for memory in results.memories
        ]
        
        return formatted_results
        
    except Exception as e:
        print(f"⚠️ Error searching memories: {e}")
        return []
        
print("✅ search_memories tool defined")

<details>
<summary>🗝️ Solution code</summary>
<br>

```python

from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from agent_memory_client.filters import UserId


class SearchMemoriesInput(BaseModel):
    """Input schema for searching memories."""
    query: str = Field(
        description="Natural language query to search for in long-term memory. "
        "Examples: 'user preferences', 'completed courses', 'learning goals'"
    )
    limit: int = Field(
        default=3,
        description="Maximum number of memories to return (default: 3)"
    )

# Get the memory client
memory_client = get_memory_client()

@tool(args_schema=SearchMemoriesInput)
async def search_memories(query: str, limit: int = 5, student_id: str = None) -> List[Dict[str, Any]]:
    """
    Search long-term memory for relevant facts and preferences.
    
    Use this tool to recall information from previous conversations,
    such as user preferences, goals, completed courses, or interests.
    
    Args:
        query: Natural language search query
        limit: Maximum number of results to return
        student_id: User identifier (passed via context)
        
    Returns:
        List of relevant memories with text content
    """
    try:
        # Validate student_id
        if not student_id:
            return []
        
        # Search long-term memories
        results = await memory_client.search_long_term_memory(
            text=query,
            user_id=UserId(eq=student_id),
            limit=limit
        )
        
        # Format results for LLM
        formatted_results = [
            {
                "text": memory.text,
                "memory_type": memory.memory_type,
                "topics": memory.topics if hasattr(memory, 'topics') else []
            }
            for memory in results.memories
        ]
        
        return formatted_results
        
    except Exception as e:
        print(f"⚠️ Error searching memories: {e}")
        return []
        
print("✅ search_memories tool defined")
```

</details>

### Test the Implementation

Before moving onto the next tool implementation, let's test the `search_memories` implementation above with the test utility:

> **Note:** This test searches for memories stored under the `test_user` ID from Stage 5. If you haven't run the Stage 5 notebook recently (or if your Redis instance was reset), we recommend running the multi-turn tests in that notebook first to seed long-term memory with extracted facts. We also run a seed file to make sure a few more long term memories are present.

In [ ]:
# Import the test utility from stage6_full_memory
from test_search_memories_tool import test_search_memories_tool, seed_test_memories

# Test your implementation
# Note: We'll use a test student ID that has some memories stored
await seed_test_memories()
await test_search_memories_tool(search_memories, student_id="test_user")

### 📌 Task 2: Storing Memories

You might wonder: if RAMS automatically extracts facts to long-term memory, why do we need a tool to store them explicitly? 

There's a few reasons. For one, we don't want to always be at the whim of automatic background extraction. When someone says "Remember that I prefer evening classes," they expect the agent to *actively* save that, not hope background extraction catches it. Having full control also gives us the ability to customize the long-term memory to our liking. We can change the storage type, add custom metadata, and ensure the information is stored exactly as we want it.

The `store_memory` tool needs to:

1. Accept text content, memory type, and optional topics
2. Validate the memory type (semantic, episodic, or message)
3. Call the memory client's `create_long_term_memories()` method
4. Return a success confirmation

Like the previous task, in the starter code, we've provided a `StoreMemoryInput` Pydantic model that defines the expected parameters. Your task is to implement the tool function that stores information to long-term memory.

<details>
<summary>🛠️ Show Implementation Details</summary>

**Step 1: Add the Tool Decorator**

Use the `@tool` decorator from LangChain with the `args_schema` parameter set to `StoreMemoryInput`. This tells LangChain how to parse and validate the tool's inputs.

**Step 2: Validate Memory Type**

Check that `memory_type` is one of the three valid types: "semantic", "episodic", or "message". If not, raise a `ValueError`. This prevents invalid data from being stored and gives clear feedback to the agent.

**Step 3: Validate Student ID**

Check if the `student_id` parameter is provided. If not, return an error message—we can't store memories without knowing whose memories they are.

**Step 4: Store Memory**

Call `memory_client.create_long_term_memories()` with a list containing a single memory dictionary:

```python
await memory_client.create_long_term_memories([{
    "text": text,
    "memory_type": memory_type,
    "topics": topics or [],
    "user_id": student_id
}])
```

The method takes a list of dictionaries, each with: `text`, `memory_type`, `topics`, and `user_id`.

**Step 5: Return Success Message**

Return a friendly confirmation message indicating the memory was stored successfully. Include a snippet of what was stored so the agent can confirm in its response to the user.

</details>

In [ ]:
from typing import Optional, List

class StoreMemoryInput(BaseModel):
    """Input schema for storing memories."""
    text: str = Field(
        description="The information to store in long-term memory. "
        "Examples: 'Student prefers online courses', 'Interested in machine learning'"
    )
    memory_type: str = Field(
        default="semantic",
        description="Type of memory: 'semantic' (facts), 'episodic' (events), or 'message' (conversations). "
        "Default is 'semantic'."
    )
    topics: Optional[List[str]] = Field(
        default=None,
        description="Optional list of topic tags for organization. "
        "Examples: ['preferences', 'interests', 'goals']"
    )

@tool(args_schema=StoreMemoryInput)
async def store_memory(
    text: str,
    memory_type: str = "semantic",
    topics: Optional[List[str]] = None,
    student_id: str = None
) -> str:
    """
    Store information in long-term memory.
    
    Use this tool when users share preferences, goals, constraints,
    or other information that should be remembered for future conversations.
    
    Args:
        text: The information to store
        memory_type: Type of memory (semantic, episodic, message)
        topics: Optional topic tags
        student_id: User identifier (passed via context)
        
    Returns:
        Success confirmation message
    """
    try:
        # Validate memory_type
        valid_types = ["semantic", "episodic", "message"]
        if memory_type not in valid_types:
            raise ValueError(f"Invalid memory_type. Must be one of: {valid_types}")
        
        # Validate student_id
        if not student_id:
            return "Error: No student_id provided"
        
        # Store memory
        await memory_client.create_long_term_memories([{
            "text": text,
            "memory_type": memory_type,
            "topics": topics or [],
            "user_id": student_id
        }])
        
        # Return success message
        return f"Successfully stored {memory_type} memory: {text[:50]}..."
        
    except Exception as e:
        return f"Error storing memory: {e}"

print("✅ store_memory function defined")

<details>
<summary>🗝️ Solution code</summary>
<br>

```python

@tool(args_schema=StoreMemoryInput)
async def store_memory(
    text: str,
    memory_type: str = "semantic",
    topics: Optional[List[str]] = None,
    student_id: str = None
) -> str:
    """
    Store information in long-term memory.
    
    Use this tool when users share preferences, goals, constraints,
    or other information that should be remembered for future conversations.
    
    Args:
        text: The information to store
        memory_type: Type of memory (semantic, episodic, message)
        topics: Optional topic tags
        student_id: User identifier (passed via context)
        
    Returns:
        Success confirmation message
    """
    try:
        # Validate memory_type
        valid_types = ["semantic", "episodic", "message"]
        if memory_type not in valid_types:
            raise ValueError(f"Invalid memory_type. Must be one of: {valid_types}")
        
        # Validate student_id
        if not student_id:
            return "Error: No student_id provided"
        
        # Store memory
        await memory_client.create_long_term_memories([{
            "text": text,
            "memory_type": memory_type,
            "topics": topics or [],
            "user_id": student_id
        }])
        
        # Return success message
        return f"Successfully stored {memory_type} memory: {text[:50]}..."
        
    except Exception as e:
        return f"Error storing memory: {e}"
```


</details>

### Test Your Implementation

Now let's test the `store_memory` implementation with the test utility:

In [ ]:
# Import the test utility
from test_store_memory_tool import test_store_memory_tool

# Test your implementation
await test_store_memory_tool(store_memory, student_id="test_user")

## Part 2: Testing Cross-Session Personalization

With both memory tools implemented, we can now demonstrate the capability that was missing in Stage 5: true cross-session personalization. Remember the frustrating scenario from the introduction where the agent forgot user preferences between sessions? Let's prove that's no longer the case.

We'll simulate a realistic user journey: Alice visits the course advisor, shares her preferences, leaves, and returns in a completely new session. In Stage 5, she would have had to repeat herself. Now, the agent should remember her from the previous conversation.

The key thing to watch in these tests is the reasoning trace. You'll see the agent actively deciding *when* to use each tool—this isn't hardcoded logic, it's the ReAct pattern in action. The agent reasons about what information it needs and selects the appropriate tool to get it.

### Test 1: Store Preferences (Session 1)

Alice is a new student visiting the course advisor for the first time. She shares her learning preferences and interests. Watch the reasoning trace to see how the agent recognizes this as information worth persisting and uses the `store_memory` tool to save it for future sessions.

In [ ]:
print("=" * 80)
print("TEST 1: Storing User Preferences (Session 1)")
print("=" * 80)

result1 = await run_agent_async(
    workflow,
    query="I prefer online courses and I'm really interested in machine learning and AI.",
    session_id="session_001",
    student_id="alice",
)

print("\n✅ Session 1 complete - preferences should be stored")

In the reasoning trace above, you should see the agent identify Alice's preferences as valuable information and explicitly store them to long-term memory. The agent might store multiple memories (e.g., one for "prefers online courses" and another for "interested in ML/AI") to keep facts atomic and searchable.

This is different from Stage 5's automatic extraction—here the agent is *actively deciding* to remember this information because the user explicitly shared preferences.

### Test 2: Retrieve Memories in New Session (Session 2)

Now for the moment of truth. Alice returns the next day in a **completely new session** (`session_002`). She asks for course recommendations without repeating her preferences. 

In Stage 5, the agent would have no choice but to ask clarifying questions—the working memory from yesterday's session is gone. But now, the agent has a tool to search long-term memory. Watch the reasoning trace to see if it thinks to use it.

In [ ]:
print("\n" + "=" * 80)
print("TEST 2: Retrieving Memories in New Session (Session 2)")
print("=" * 80)

result2 = await run_agent_async(
    workflow,
    query="What courses would you recommend for me?",
    session_id="session_002",  # DIFFERENT SESSION
    student_id="alice",        # SAME STUDENT
)

print("\n✅ Session 2 complete - agent should have used stored preferences")

This is cross-session personalization in action. Notice the multi-step reasoning:

1. **Thought**: "The user asked for recommendations but didn't specify criteria. Let me check if I have any stored information about their preferences."
2. **Action**: `search_memories` with a query about preferences
3. **Observation**: Finds Alice's stored preferences (online courses, ML/AI interest)
4. **Thought**: "Now I can search for courses that match these preferences."
5. **Action**: `search_courses` with personalized criteria

The agent remembered Alice without her having to repeat herself. This is the seamless experience we were aiming for.

### Test 3: Multi-Tool Orchestration

Let's push the agent further with a query that requires chaining multiple tools together. This tests not just memory retrieval, but the agent's ability to plan a multi-step approach: recall preferences, search courses, and filter by prerequisites—all in one turn.

In [ ]:
print("\n" + "=" * 80)
print("TEST 3: Multi-Tool Orchestration")
print("=" * 80)

result3 = await run_agent_async(
    workflow,
    query="Show me courses that match my interests and tell me which ones have prerequisites.",
    session_id="session_003",
    student_id="alice",
)

print("\n✅ Multi-tool test complete")

The reasoning trace here reveals sophisticated planning. The agent doesn't just blindly execute tools—it forms a strategy:

- First, understand what "my interests" means by searching memories
- Then, find courses matching those interests
- Finally, analyze prerequisite information from the results

This multi-step orchestration is the ReAct pattern at its best: the agent reasons, acts, observes the result, and decides what to do next. Each iteration brings it closer to a complete answer.

## Part 3: Understanding Tool Orchestration Patterns

Now that you've seen the agent in action, let's step back and examine the patterns that emerge. Different types of queries lead to different tool orchestration flows. Understanding these patterns helps you predict agent behavior and design better tools.

**Pattern 1: Store First, Then Search**  
When users share new information *and* ask a question in the same turn, the agent typically stores first, then searches. This ensures the new preference is persisted before moving on.  
*Example:* "I like Python programming. Show me related courses."  
*Flow:* `store_memory` → `search_courses` → FINISH

**Pattern 2: Search Memories First**  
When users ask personalized questions in a new session (without providing new context), the agent searches memories to recall who they are and what they prefer.  
*Example:* "What courses fit my preferences?"  
*Flow:* `search_memories` → `search_courses` → FINISH

**Pattern 3: Search Courses Only**  
For factual, non-personalized queries, the agent goes straight to course search. No memory tools needed—the question is self-contained.  
*Example:* "What is CS401?"  
*Flow:* `search_courses` → FINISH

**Pattern 4: Complex Multi-Tool**  
Some queries require all three tools: store a new preference, recall existing context, and search for matching content.  
*Example:* "I want to learn databases. Remember that and find relevant courses."  
*Flow:* `store_memory` → `search_memories` → `search_courses` → FINISH

Let's run through a few patterns with a new student (Charlie) to see these flows in action:

In [ ]:
print("=" * 80)
print("PATTERN TESTING")
print("=" * 80)

patterns = [
    ("Pattern 1", "I like Python programming. Show me related courses.", "charlie", "charlie_001"),
    ("Pattern 2", "What courses fit my preferences?", "charlie", "charlie_002"),
    ("Pattern 3", "What is CS401?", "charlie", "charlie_003"),
]

for pattern_name, query, student_id, session_id in patterns:
    print(f"\n--- {pattern_name} ---")
    print(f"Query: {query}")
    await run_agent_async(
        workflow,
        query=query,
        session_id=session_id,
        student_id=student_id,
    )
    print("\n")

## Wrap Up 🏁

You've completed Stage 6 and built a production-ready agent with full memory capabilities. Let's reflect on what you accomplished:

**The Problem We Solved**

At the start of this notebook, we identified a frustrating limitation: our Stage 5 agent had amnesia between sessions. RAMS was automatically extracting facts to long-term memory, but the agent had no way to access them. Users had to repeat their preferences every time they started a new conversation.

**The Solution We Built**

You implemented two tools that unlock long-term memory for the agent:

- **`search_memories`**: Enables the agent to query stored facts, preferences, and history from previous sessions. When a user asks "What courses fit my preferences?" in a new session, the agent can now recall who they are.

- **`store_memory`**: Gives the agent explicit control over what gets remembered. When a user says "Remember that I prefer evening classes," the agent actively stores that fact rather than relying on automatic extraction.

**The Patterns We Discovered**

Through testing, you observed how the agent orchestrates these tools dynamically:
- Storing information when users share preferences
- Searching memories before personalized recommendations  
- Chaining multiple tools for complex queries
- Skipping memory tools entirely for factual questions

This isn't hardcoded logic—it's the ReAct pattern enabling the agent to reason about what information it needs and select the right tool to get it.

---

### The Complete Journey

You've now completed the entire progressive agents learning path. Here's the full architecture you've built from the ground up:

| Section | Stage | What You Built |
|---------|-------|----------------|
| **Context Engineering Foundations** | Stage 1 | Baseline RAG with retrieved context + generation |
| | Stage 2 | Data-engineered RAG with structured data and optimized chunking |
| **From RAG to Agent** | Stage 3 | Hierarchical retrieval with intent classification and progressive disclosure |
| | Stage 4 | Hybrid search + ReAct with visible reasoning and tool orchestration |
| **Memory & Context** | Stage 5 | Working memory for multi-turn conversation continuity |
| | Stage 6 | Full memory with explicit long-term memory control |

Each stage built on the previous one, and together they form a complete context engineering system:

- **Context engineering** taught you how to structure, chunk, and assemble information for optimal LLM consumption
- **Intelligent retrieval** gave you multiple strategies (semantic, exact match, hybrid) to find the right information
- **Agentic reasoning** enabled visible decision-making through the ReAct pattern
- **Memory management** provided both automatic extraction and explicit control over what the agent remembers

---

### Congratulations! 🎉

You've mastered the basics of context engineering for AI agents. The concepts and patterns you've learned—hierarchical context assembly, hybrid search strategies, ReAct reasoning, and two-tier memory management—are the building blocks of sophisticated, context-aware AI systems.

Now go build something amazing.